# Ejercicio 1 - Consumir una API.

Se extraerán los datos de la página _[https://www.alphavantage.co/](https://www.alphavantage.co/)_

Dicha página ofrece información y cotizaciones sobre la bolsa.
Posee una API documentada en [https://www.alphavantage.co/documentation/](https://www.alphavantage.co/documentation/)
La API de "Alpha Vantage" permite llamar a algunas de sus funciones mediante una clave llamada "demo". Para levantar
las limitaciones deberemos registrarnos y obtener una clave nueva. La clave puede obtenerse fácilmente desde [https://www.alphavantage.co/support/#api-key](https://www.alphavantage.co/support/#api-key).
En el ejercicio obtendremos el precio de la cotización
de dos empresas distintas una con la clave "demo" y otra con la clave facilitada por "Alpha Vantage".

La solicitud elegida es: <https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&outputsize=full&apikey=demo>

Describimos los parámetros:

function=TIME_SERIES_INTRADAY -> Remite, en formato JSON, una serie temporal con los precios de cotización de la empresa especificada.

symbol=IBM -> Se selecciona la empresa deseada.

interval=5min                 -> Los valores se muestran en el intervalo seleccionado: 1min, 5min, 15min, 30min, 60min.

outputsize=full  ->  La opción "compact" envía 100 registros. "full" envía 30 días.

apikey=demo -> Clave que permite el acceso.

#### A continuación el código.

Primero importamos las librerías necesarias y definimos las claves como variables globales:

In [13]:
# El script accede a la API de "https://www.alphavantage.co/".
# Nótese que el dominio no es ".com" si no ".co"

# Importamos la librerías necesarias.
import requests # Para la solicitud http.
import json     # La respuesta se da en formato JSON.

# La clave "demo" nos permite tipos limitados de consultas.
API_KEY_DEMO = f"demo"

# El registro a "https://www.alphavantage.co/" permite mejor acceso mediante la siguiente clave:
API_KEY = f"G0JZT4C79E2OKFRE"

La siguiente función construye la URL, envía la solicitud y devuelve el precio de la cotización a la obertura:

In [15]:
# Función para obtener datos de cotizaciones.
def get_price (symbol, interval, outputsize):

    # Construimos la llamada "GET".
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=" + symbol
    url = url + f"&interval=" + interval
    url = url + f"&outputsize=" + outputsize
    url = url + f"&apikey=" + KEY

    # Imprimimos la URL.
    print ("url: " + url)

    # Mandamos la solicitud y almacenamos la respuesta.
    response = requests.get(url)

    # Comprobamos si la respuesta es un éxito.
    if response.status_code == 200:

        # Filtramos el dato deseado, el precio.
        data = response.json()
        last_refreshed = data["Meta Data"]["3. Last Refreshed"]
        ts = "Time Series (" + interval + ")"
        price = data[ts][last_refreshed]["1. open"]

        return price
    else:
        return None

Obtenemos la cotización de la empresa "IBM" con la clave "demo", este código produce la salida deseada:

In [17]:
# Main

# Con la clave de "demo" solo podemos consultar a "IBM" con las
# opciones "5min" y "full".
KEY = API_KEY_DEMO

company=f"IBM"
price = get_price (company, f"5min", f"full")

if price is not None:
    print(f"{company}: {price}")

url: https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&outputsize=full&apikey=demo
IBM: 220.5000


Finalmente obtenemos la cotización de "ST Microelectronics" con la clave obtenida durante el registro:

In [19]:
# Después de habernos registrado podemos consultar los precios de cualquier
# compañía y variar las opciones de la solicitud.
KEY = API_KEY

# Elegimos la compañía "ST Microelectronics"
company=f"STM"
price = get_price (company, f"60min", f"compact")

if price is not None:
     print(f"{company}: {price}")

url: https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=STM&interval=60min&outputsize=compact&apikey=G0JZT4C79E2OKFRE
STM: 30.3600


# Ejercicio 2 - Obtener datos con WEB scraping.

Para este ejercicio accederemos a [https://www.google.com/finance](https://www.google.com/finance) y obtendremos los datos de cotización en bolsa de la empresa "IBM". Para ello debemos acceder a la dirección [https://www.google.com/finance/quote/IBM:NYSE?hl=en](https://www.google.com/finance/quote/IBM:NYSE?hl=en).

Comprobamos el archivo [https://www.google.com/robots.txt](https://www.google.com/robots.txt). En él encontramos la entrada
"Allow: /finance".
La página permite pues la extracción de sus datos.

#### Paso 1 - Inspeccionar el código.

 Examinando la WEB comprobamos que hay una sección con un gráfico con las cotizaciones. A su derecha vemos una caja de texto con etiquetas "PREVIOUS CLOSE", "DAY RANGE", "MARKET CAP", "AVG VOLUME", etc. Son los datos de interes.

Inspeccionando el código fuente de la página descubrimos que, en cada linea de la tabla, se encuentra en un elemento de la clase:

\<div class="gyFHrc"\>...\</div\>

Dentro de dicho elemento encontramos los tags:

\<div class="mfs7Fc"\>...\</div\>
\<div class="P6K39c"\>$223.43\</div\>

que son los deseados. Tomamos nota de las clases de los tags para pasárselos al parser de HTML. Nótese que la tabla contiene varias filas, luego encontraremos varios tags de dichas clases.

#### Paso 2 - Scrape.

En este paso empezamos con el código.

En primer lugar importamos los módulos necesarios:

In [33]:
import requests
from bs4 import BeautifulSoup

La siguiente función nos facilita la construcción de la URL:

In [35]:
# Construimos la URL a inspeccionar:

def set_URL (INDEX, SYMBOL):
    BASE_URL = "https://www.google.com/finance"
    LANGUAGE = "en"
    return f"{BASE_URL}/quote/{SYMBOL}:{INDEX}?hl={LANGUAGE}"

Ahora se realiza el "scrape" propiamente dicho, también queda definido como una función:

In [37]:
def scrape (page):
    # Utilizamos un parser HTML para obtener el contenido
    s = BeautifulSoup(page.content, "html.parser")
    return s

#### Paso 3 - Parsing.

En este paso obtenemos los elementos que hemos descubierto en la inspección de la WEB, mediante su función:

In [40]:
def parse (s):
    # Después de inspeccionar la página descubrimos que los items que describen
    # la cotización están en la clase "gyFHrc".
    items = s.find_all("div", {"class": "gyFHrc"})

    # Creamos un diccionario para almacenar la descripción de la contización.
    stock_description = {}

    # Iteramos para cada uno de los elementos del diccionario.
    for item in items:
        item_description = item.find("div", {"class": "mfs7Fc"}).text
        item_value = item.find("div", {"class": "P6K39c"}).text
        stock_description[item_description] = item_value
    return stock_description

Llamamos a las funciones anteriores para recoger los datos en la rutina principal:

Primero con la empresa "IBM". Debe indicarse también el índice, en nuestro caso "NYSE":

In [43]:
# Main

# Construimos y leemos la URL de la página WEB.
url = set_URL ("NYSE", "IBM")
page = requests.get(url)

print ("url: " + url)
print ("status: %d" % page.status_code)

# Obtenemos los datos de cotización mediante "scrape".
scr = scrape (page)
stock = parse (scr)
print(stock)

url: https://www.google.com/finance/quote/IBM:NYSE?hl=en
status: 200
{'Previous close': '$223.43', 'Day range': '$220.77 - $224.15', 'Year range': '$135.87 - $224.15', 'Market cap': '203.43B USD', 'Avg Volume': '3.59M', 'P/E ratio': '24.34', 'Dividend yield': '3.02%', 'Primary exchange': 'NYSE', 'CEO': 'Arvind Krishna', 'Founded': 'Jun 16, 1911', 'Headquarters': 'Armonk, New YorkUnited States', 'Website': 'ibm.com', 'Employees': '282,200'}


Luego la empresa "ST Microelectronics":

In [45]:
# Construimos y leemos la URL de la página WEB.
url = set_URL ("NYSE", "STM")
page = requests.get (url)

print ("url: " + url)
print ("status: %d" % page.status_code)

# Obtenemos los datos de cotización mediante "scrape".
scr = scrape (page)
stock = parse (scr)
print(stock)

url: https://www.google.com/finance/quote/STM:NYSE?hl=en
status: 200
{'Previous close': '$29.48', 'Day range': '$30.32 - $30.75', 'Year range': '$27.45 - $51.27', 'Market cap': '27.07B USD', 'Avg Volume': '4.69M', 'CEO': 'Jean-Marc Chery', 'Founded': '1987', 'Headquarters': 'Geneva, GenevaSwitzerland', 'Website': 'st.com', 'Employees': '51,323'}
